In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/g-research-crypto-forecasting/example_sample_submission.csv
/kaggle/input/g-research-crypto-forecasting/asset_details.csv
/kaggle/input/g-research-crypto-forecasting/example_test.csv
/kaggle/input/g-research-crypto-forecasting/train.csv
/kaggle/input/g-research-crypto-forecasting/supplemental_train.csv
/kaggle/input/g-research-crypto-forecasting/gresearch_crypto/competition.cpython-37m-x86_64-linux-gnu.so
/kaggle/input/g-research-crypto-forecasting/gresearch_crypto/__init__.py


## Data loading

In [2]:
data_train = '/kaggle/input/g-research-crypto-forecasting/train.csv'
data_asset_details = '/kaggle/input/g-research-crypto-forecasting/asset_details.csv'
data_supplemental_train = '/kaggle/input/g-research-crypto-forecasting/supplemental_train.csv'

In [3]:
import random
import pandas as pd
import numpy as np
import lightgbm as lgb
import time
import datetime
import plotly.graph_objects as go

In [4]:
df_train = pd.read_csv(data_train, 
                       dtype={'Asset_ID': 'int8', 'Count': 'int32', 'row_id': 'int32', 'Count': 'int32', 
                              'Open': 'float64', 'High': 'float64', 'Low': 'float64', 'Close': 'float64', 
                              'Volume': 'float64', 'VWAP': 'float64'
                             }
                      )
df_train.head()

,timestamp,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,Target
0,1514764860,2,40,2376.5800,2399.5000,2357.1400,2374.5900,19.233005,2373.116392,-0.004218
1,1514764860,0,5,8.5300,8.5300,8.5300,8.5300,78.380000,8.530000,-0.014399
2,1514764860,1,229,13835.1940,14013.8000,13666.1100,13850.1760,31.550062,13827.062093,-0.014643
3,1514764860,5,32,7.6596,7.6596,7.6567,7.6576,6626.713370,7.657713,-0.013922
4,1514764860,7,5,25.9200,25.9200,25.8740,25.8770,121.087310,25.891363,-0.008264


In [5]:
import gresearch_crypto

In [6]:
df_assets = pd.read_csv(data_asset_details).sort_values(by='Asset_ID')
df_assets.head()

,Asset_ID,Weight,Asset_Name
1,0,4.304065,Binance Coin
2,1,6.779922,Bitcoin
0,2,2.397895,Bitcoin Cash
10,3,4.406719,Cardano
13,4,3.555348,Dogecoin


In [7]:
df_assets

,Asset_ID,Weight,Asset_Name
1,0,4.304065,Binance Coin
2,1,6.779922,Bitcoin
0,2,2.397895,Bitcoin Cash
10,3,4.406719,Cardano
13,4,3.555348,Dogecoin
3,5,1.386294,EOS.IO
5,6,5.894403,Ethereum
4,7,2.079442,Ethereum Classic
11,8,1.098612,IOTA
6,9,2.397895,Litecoin


## Data preprocessing

In [8]:
from lightgbm import LGBMRegressor

In [9]:
def upper_shadow(df):
    return df['High'] - np.maximum(df['Close'], df['Open'])

def lower_shadow(df):
    return np.minimum(df['Close'], df['Open']) - df['Low']

def get_features(df):
    df_feat = df[['Count', 'Open', 'High', 'Low', 'Close', 'Volume', 'VWAP']].copy()
    df_feat['Upper_Shadow'] = upper_shadow(df_feat)
    df_feat['Lower_Shadow'] = lower_shadow(df_feat)
    
    df_feat["Close/Open"] = df_feat["Close"] / df_feat["Open"] 
    df_feat["Close-Open"] = df_feat["Close"] - df_feat["Open"] 
    df_feat["High-Low"] = df_feat["High"] - df_feat["Low"] 
    df_feat["High/Low"] = df_feat["High"] / df_feat["Low"]
    
    df_feat['Mean'] = df_feat[['Open', 'High', 'Low', 'Close']].mean(axis=1)


    df_feat["Median"] = df_feat[["Open", "High", "Low", "Close"]].median(axis=1)
    return df_feat

In [10]:
from sklearn.preprocessing import StandardScaler

In [11]:
from sklearn.metrics import f1_score
import pandas as pd
from skopt import gp_minimize
from skopt.space import Real, Integer
from sklearn.model_selection import train_test_split
from functools import partial

In [12]:
def m(x, w):
    """Weighted Mean"""
    return np.sum(x * w) / np.sum(w)
def cov(x, y, w):
    return np.sum(w * (x - m(x, w)) * (y - m(y, w))) / np.sum(w)

def wcc(x, y, w):
    return cov(x, y, w) / np.sqrt(cov(x, x, w) * cov(y, y, w))

## Parameters grid

In [13]:
space = [
    Real(0.5, 0.8, name="colsample_bytree"),
    Real(0.001, 0.5, name="learning_rate"),
    Integer(6, 15, name="max_depth"),
    Integer(100, 1000, name="n_estimators"),
    Real(0.5, 0.95, name="subsample"),
]

In [14]:
train_ratio = 0.75

In [15]:
df_train['Asset_ID'].value_counts()

1     1956282
6     1956200
9     1956030
5     1955140
2     1953537
7     1951127
0     1942619
13    1874560
3     1791867
12    1778749
11    1701261
8     1592071
4     1156866
10     670497
Name: Asset_ID, dtype: int64

In [16]:
weight = df_assets['Weight'].values[0]

In [17]:
df = df_train.reset_index(drop=True)

In [18]:
df_assets.shape

(14, 3)

## Best params finding

In [19]:
def return_model_assessment(args, X_train, y_train, X_test, w):
    global models, train_scores, test_scores, curr_model_hyper_params
    params = {curr_model_hyper_params[i]: args[i] for i, j in enumerate(curr_model_hyper_params)}
    model = LGBMRegressor(random_state=2022)
    model.set_params(**params)
    fitted_model = model.fit(X_train, y_train, sample_weight=None)
    models.append(fitted_model)
    train_predictions = model.predict(X_train)
    test_predictions = model.predict(X_test)
    train_score = wcc(train_predictions, y_train, w)
    test_score = wcc(test_predictions, y_test, w)
    train_scores.append(train_score)
    test_scores.append(test_score)
    return 1 - test_score

In [20]:
for i in range(10, 11):
    print(i, '++++++++++++++++++++++++++')
    df_assets_copy = df_assets.copy()
    weight = df_assets_copy[df_assets_copy['Asset_ID'] == i]['Weight'].values[0]
    df_copy = df_train.copy()
    df_copy = df_copy[df_copy['Asset_ID'] == i].copy()
    df = df_copy.reset_index(drop=True)
    df_proc = get_features(df)
    df_proc['y'] = df['Target']
    df_proc = df_proc.dropna(how="any")

    df_proc = df_proc.reset_index()
    X = df_proc.drop(labels=np.where(np.isinf(df_proc))[0], axis=0)
    X = X.reset_index()
    
    y = X["y"]
    X = X.drop("y", axis=1)
    
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    
    col_names = df_proc.drop('y', axis=1).columns
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1 - train_ratio, random_state=42)
    target_col = "y"
    
    models = []
    train_scores = []
    test_scores = []
    curr_model_hyper_params = ['colsample_bytree', 'learning_rate',
                               'max_depth', 'n_estimators', 'subsample']
    objective_function = partial(return_model_assessment, X_train=X_train, y_train=y_train, X_test=X_test, w=weight)

    n_calls = 5 
    results = gp_minimize(objective_function, space, base_estimator=None, n_calls=5, n_random_starts=n_calls-1, random_state=42)
    
    print("""Best parameters:
    - colsample_bytree=%f
    - learning_rate=%.6f
    - max_depth=%d
    - n_estimators=%d
    - subsample=%f""" % (results.x[0], results.x[1],
                         results.x[2], results.x[3],
                         results.x[4]))

10 ++++++++++++++++++++++++++
Best parameters:
    - colsample_bytree=0.685244
    - learning_rate=0.306215
    - max_depth=6
    - n_estimators=121
    - subsample=0.736149


## Graphic plot (for single version)

In [21]:
import plotly.express as px
metrics = pd.DataFrame(train_scores + test_scores)
metrics.loc[:,'dataset'] = ["train_score"]*n_calls + ["test_score"]*n_calls
metrics.loc[:,'Iteration Number'] = list(range(1,n_calls+1)) + list(range(1,n_calls+1))
metrics.columns = ["WCC", "dataset", "Iteration Number"]
fig = px.line(metrics, x="Iteration Number", y="WCC", color="dataset")
fig.show()

In [22]:
print("""Best parameters:
- colsample_bytree=%f
- learning_rate=%.6f
- max_depth=%d
- n_estimators=%d
- subsample=%f""" % (results.x[0], results.x[1],
                            results.x[2], results.x[3],
                            results.x[4]))

Best parameters:
- colsample_bytree=0.685244
- learning_rate=0.306215
- max_depth=6
- n_estimators=121
- subsample=0.736149


    Real(0.5, 0.8, name="colsample_bytree"),
    Real(0.001, 0.5, name="learning_rate"),
    Integer(6, 15, name="max_depth"),
    Integer(100, 1000, name="n_estimators"),
    Real(0.5, 0.95, name="subsample"),

In [23]:
results.x

[0.685244452888315, 0.3062149270836522, 6, 121, 0.7361485971162751]